In [5]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
import time, os, random, sys
import math
import hyperopt.tpe
import hpsklearn.components
import hpsklearn.demo_support
import warnings

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1500)

warnings.filterwarnings('ignore')
np.random.seed(1)

In [9]:
def xgboost_lr(File):
    
    start = time.clock()
    train_df, test_df = train_test_split(File, train_size = 0.8)
    X_train = train_df.drop(train_df.columns[0], axis=1)
    y_train = train_df[train_df.columns[0]]
    X_test = test_df.drop(test_df.columns[0], axis=1)
    y_test = test_df[test_df.columns[0]]
    
    
    dtrain = xgb.DMatrix(X_train.values,y_train.values)
    dtest = xgb.DMatrix(X_test.values, y_test.values)
    param = {'max_depth':7, 'silent': 1, 'objective':'binary:logistic'}
    watchlist = [(dtest, 'eval'), (dtrain, 'train')]
    num_round = 30

    bst= xgb.train(param, dtrain, num_round)

    y_pred = bst.predict(dtrain, pred_leaf=True)
    num_leaf= 1000
    X_train_leaves = np.zeros([len(y_pred),len(y_pred[0]) * num_leaf],dtype=np.int64)
    for i in range(0,len(y_pred)):
        temp = np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[i])
        X_train_leaves[i][temp] += 1

    y_pred= bst.predict(dtest, pred_leaf=True)    
    X_test_leaves = np.zeros([len(y_pred),len(y_pred[0]) * num_leaf],dtype=np.int64)
    for i in range(0,len(y_pred)):
        temp = np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[i])
        X_test_leaves[i][temp] += 1

    lm = LogisticRegression(penalty='l2',C=0.1) # logestic model construction
    lm.fit(X_train_leaves,y_train)  # fitting the data

    y_pred_est = lm.predict_proba(X_test_leaves)   # Give the probabilty on each label

    gbdtlr_auc1 = roc_auc_score(y_test, y_pred_est[:,1])
    print('XGBDT+LR auc 1: %.5f' % gbdtlr_auc1)
    
    lr = LogisticRegression(n_jobs=-1)
    X_train_ext = hstack([X_train_leaves, X_train])
    lr.fit(X_train_ext, y_train)
    X_test_ext = hstack([X_test_leaves, X_test])
    y_pred_gbdtlr2 = lr.predict_proba(X_test_ext)[:, 1]
    gbdtlr_auc2 = roc_auc_score(y_test, y_pred_gbdtlr2)
    print('XGBDT+LR auc 2: %.5f' % gbdtlr_auc2)
    f_time =time.clock()-start
    print('XGBDT+LR time taken: %.2f'% f_time)
    
    #+NB

    
    start = time.clock()
    gnb= GaussianNB()

    gnb.fit(X_train_leaves, y_train)
    Y_pred_nb=gnb.predict_proba(X_test_leaves)[:,1]
    gnb_auc = roc_auc_score(y_test,Y_pred_nb)
    print('GBDT + GNB auc: %.5f'% gnb_auc)    
    '''    
    gnb.fit(X_train_ext, y_train)
    y_pred_gnb2=gnb.predict_proba(X_test_ext)[:,1]
    gnb_auc2=roc_auc_score(y_test,y_pred_gnb2)
    print('GNB auc2: %.5f' % gnb_auc2)
    '''
    '''#svc
    svc=SVC(probability=True)
    svc.fit(X_train_leaves, y_train)
    Y_pred_svc=svc.predict_proba(X_test_leaves)[:,1]
    svc_auc=roc_auc_score(y_test,Y_pred_svc)
    print('GBDT + SVC auc: %.5f' % svc_auc)
    
    svc.fit(X_train_ext, y_train)
    y_pred_svc2=svc.predict_proba(X_test_ext)[:, 1]
    svc_auc2=roc_auc_score(y_test,y_pred_svc2)
    print('GBDT + SVC auc2: %.5f' % svc_auc2)'''
    
    '''#KNN
    knn=KNeighborsClassifier(n_neighbors = 3)
    knn.fit(X_train_leaves, y_train)
    Y_pred_knn=knn.predict_proba(X_test_leaves)[:,1]
    knn_auc=roc_auc_score(y_test,Y_pred_knn)
    print('GBDT + KNN auc : %.5f' % knn_auc)
    
    knn.fit(X_train_ext, y_train)
    y_pred_knn2=knn.predict_proba(X_test_ext)[:, 1]
    knn_auc2=roc_auc_score(y_test,y_pred_knn2)
    print('GBDT + KNN auc2: %.5f' % knn_auc2)
    '''
    #perceptron

    perceptron = Perceptron()
    perceptron.fit(X_train_leaves, y_train)
    y_pred_perc=perceptron.predict(X_test_leaves)
    perc_auc=roc_auc_score(y_test,y_pred_perc)
    print('GBDT + Perceptron auc : %.5f' % perc_auc)
    
    perceptron.fit(X_train_ext, y_train)
    y_pred_perc2=perceptron.predict(X_test_ext)
    perc_auc2=roc_auc_score(y_test,y_pred_perc2)
    print('GBDT + Perceptron auc2 : %.5f' % perc_auc2 )
    
    #linear svc

    lin = LinearSVC()
    lin.fit(X_train_leaves, y_train)
    y_pred_lin=lin.predict(X_test_leaves)
    lin_auc=roc_auc_score(y_test,y_pred_lin)
    print('GBDT + Linear SVC auc : %.5f' % lin_auc)
    
    lin.fit(X_train_ext, y_train)
    y_pred_lin2=lin.predict(X_test_ext)
    lin_auc2=roc_auc_score(y_test,y_pred_lin2)
    print('GBDT + Linear SVC auc2 : %.5f' % lin_auc2)
    
    #SGD

    sgd = SGDClassifier(loss='log')
    sgd.fit(X_train_leaves, y_train)
    Y_pred_sgd=sgd.predict_proba(X_test_leaves)[:, 1]
    sgd_auc=roc_auc_score(y_test,Y_pred_sgd)
    print('GBDT + SGD auc : %.5f' % sgd_auc)
    
    sgd.fit(X_train_ext, y_train)
    Y_pred_sgd2=sgd.predict_proba(X_test_ext)[:, 1]
    sgd_auc2=roc_auc_score(y_test,Y_pred_sgd2)
    print('GBDT + SGD auc2 : %.5f' % sgd_auc2)
    f_time =time.clock()-start
    print('GBDT + other classfier time taken: %.2f'% f_time)    
    
    #XGB
    start = time.clock()
    xgbc=XGBClassifier()
    xgbc.fit(X_train,y_train)
    Y_pred_xgb=xgbc.predict_proba(X_test)[:,1]
    xgb_auc= roc_auc_score(y_test,Y_pred_xgb)
    print('XGB auc : %.5f' % xgb_auc)
    
    '''#XGB with leaves

    xgbc.fit(X_train_leaves,y_train)
    Y_pred_xgb=xgbc.predict(X_test_leaves)
    xgb_auc= roc_auc_score(y_test,Y_pred_xgb)
    print('GBDT + XGB auc: %.5f' % xgb_auc)
    
    #XGB with features ext
    
    xgbc.fit(X_train_ext, y_train)
    y_pred_xgb2=xgbc.predict(X_test_ext)
    xgb_auc2=roc_auc_score(y_test,y_pred_xgb2)
    print('GBDT + XGB auc2: %.5f' %xgb_auc2)
    
    f_time =time.clock()-start
    print('XGB time taken: %.2f'% f_time)'''
    
    #lightGBM
    start = time.clock()
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    
    
    
    # specify your configurations as a dict
    params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
    
    }

    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=20,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=5,
                    verbose_eval=False)



    y_pred_lgb = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    # eval
    lgb_auc=roc_auc_score(y_test,y_pred_lgb)
    print('lightGBM auc : %.5f' % lgb_auc)
    
    lgb_train=lgb.Dataset(X_train_leaves, y_train)
    lgb_eval=lgb.Dataset(X_test_leaves, y_test, reference=lgb_train)
    gbm = lgb.train(params,
                  lgb_train,
                    num_boost_round=20,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=5,
                   verbose_eval=False)
    y_pred_lgb2 =gbm.predict(X_test_leaves, num_iteration=gbm.best_iteration)
    lgb_auc2=roc_auc_score(y_test, y_pred_lgb2)
    
    print('GBDT + lightGBM auc : %.5f' % lgb_auc2)
    
    lgb_train=lgb.Dataset(X_train_ext, y_train)
    lgb_eval=lgb.Dataset(X_test_ext, y_test, reference=lgb_train)
    gbm = lgb.train(params,
                  lgb_train,
                    num_boost_round=20,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=5,
                   verbose_eval=False)
    y_pred_lgb3 =gbm.predict(X_test_ext, num_iteration=gbm.best_iteration)
    lgb_auc3=roc_auc_score(y_test, y_pred_lgb3)
    
    print('GBDT + lightGBM auc2 : %.5f' % lgb_auc3)
    
    
    f_time=time.clock()-start
    print('lightGBM time taken: %.2f'% f_time)

In [3]:
example = pd.read_csv('example.csv')

In [10]:
xgboost_lr(example)

XGBDT+LR auc 1: 0.94069
XGBDT+LR auc 2: 0.62187
XGBDT+LR time taken: 6.23
GBDT + GNB auc: 0.86978
GBDT + Perceptron auc : 0.79588
GBDT + Perceptron auc2 : 0.48035
GBDT + Linear SVC auc : 0.85587
GBDT + Linear SVC auc2 : 0.46910
GBDT + SGD auc : 0.92947
GBDT + SGD auc2 : 0.52675
GBDT + other classfier time taken: 14.77
XGB auc : 0.93379
lightGBM auc : 0.93402
GBDT + lightGBM auc : 0.93368
GBDT + lightGBM auc2 : 0.93126
lightGBM time taken: 2.91


In [11]:
xgboost_lr(example)

XGBDT+LR auc 1: 0.92607
XGBDT+LR auc 2: 0.56891
XGBDT+LR time taken: 4.43
GBDT + GNB auc: 0.86347
GBDT + Perceptron auc : 0.85147
GBDT + Perceptron auc2 : 0.50000
GBDT + Linear SVC auc : 0.84385
GBDT + Linear SVC auc2 : 0.50617
GBDT + SGD auc : 0.91063
GBDT + SGD auc2 : 0.49945
GBDT + other classfier time taken: 13.48
XGB auc : 0.93151
lightGBM auc : 0.92760
GBDT + lightGBM auc : 0.92417
GBDT + lightGBM auc2 : 0.91973
lightGBM time taken: 3.01


In [12]:
xgboost_lr(example)

XGBDT+LR auc 1: 0.93185
XGBDT+LR auc 2: 0.54976
XGBDT+LR time taken: 4.46
GBDT + GNB auc: 0.85293
GBDT + Perceptron auc : 0.80626
GBDT + Perceptron auc2 : 0.51418
GBDT + Linear SVC auc : 0.82021
GBDT + Linear SVC auc2 : 0.50000
GBDT + SGD auc : 0.91307
GBDT + SGD auc2 : 0.50317
GBDT + other classfier time taken: 13.79
XGB auc : 0.92898
lightGBM auc : 0.92622
GBDT + lightGBM auc : 0.92848
GBDT + lightGBM auc2 : 0.93084
lightGBM time taken: 2.94
